In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# Import KMM method form adapt.instance_based module
from adapt.instance_based import KMM
import json


# with open('matbench_expt_is_metal.json', 'r') as json_file:
#     # Load the JSON data
#     data = json.load(json_file)

# df = pd.DataFrame(data['data'], columns = data['columns'])
# df = df.rename(columns={'composition': 'formula'})

In [2]:
# Define the file path
file_path = 'glass_target_clusters50.json'
ncluster=50
featurefile = 'glass_features.csv'

In [3]:
from matminer.featurizers.conversions import StrToComposition
import os

if not os.path.exists(featurefile):
    df1 = StrToComposition().featurize_dataframe(df, "formula")
    df1.head()

    from matminer.featurizers.composition import ElementProperty

    ep_feat = ElementProperty.from_preset(preset_name="magpie")
    df2 = ep_feat.featurize_dataframe(df1, col_id="composition")  # input the "composition" column to the featurizer
    df2.head()

    #y = df2['gfa'].values
    excluded = ["formula", "composition", "formula", "gfa"]

    #X = df2.drop(excluded, axis=1)
    df2.to_csv(featurefile)
else:  #load the features
    df2 = pd.read_csv(featurefile)


In [6]:
y = df2['gfa'].values
excluded = ["formula", "composition", "formula", "gfa"]
X = df2.drop(excluded, axis=1)

In [7]:
# print(X.head)
# print(X.shape)
# print(X.iloc[0])
# print(y)

In [8]:
# from pymatgen.core import Composition

# target_id_list = []
# for i, s in enumerate(df2["formula"]):
#     if "Cu" in Composition(s).as_dict().keys() and "Li" in Composition(s).as_dict().keys():
#         target_id_list.append(i)
# print(target_id_list)

In [9]:


# Read the JSON file
with open(file_path, 'r') as file:
    cluster_ids = json.load(file)

# Access the data from the JSON file
target_id_list = cluster_ids["2"]
print(target_id_list)

[4438, 4442, 4443, 4444, 4449, 4455, 4464, 4498, 4513, 4524, 4525, 4536, 4537, 4538, 4539, 4540, 4541, 4542, 4543, 4544, 4545, 4546, 4547, 4548, 4549, 4550, 4551, 4552, 4553, 4554, 4555, 4556, 4557, 4559, 4561, 4562, 4563, 4564, 4565, 4747, 4749, 4750, 4751, 4752, 4753, 4754, 4755, 4756, 4757, 4758, 4761, 4769, 4770, 4771, 4772, 4775, 4776, 4777, 4779, 4781, 4783, 4785, 4786, 4787, 4789, 4790, 4791, 4792, 4793, 4801, 4802, 4803, 4806, 4808, 4817, 4822, 4825, 4826, 4827, 4838, 4839, 4843, 4844, 4845, 4846, 4847, 4848, 4849, 4852, 4853, 4866, 4867, 4868, 4870, 4873, 4874, 4882, 4884, 4885, 4886, 4888, 4893, 4896, 4898, 4899, 4905, 4906, 4907, 4908, 4910, 4913, 4923, 4927, 4928, 4929, 4933, 4934, 4938, 4939, 4940, 4941, 4943, 4945, 4949, 4953, 4955, 4957, 4958, 4961, 4962, 4963, 4964, 4965, 4972, 4973, 4974, 4976, 4977, 4978, 4982, 4983, 4986, 4987, 4989, 4990, 4991, 4992, 4993, 4994, 5001, 5002, 5004, 5008, 5009, 5013, 5023, 5040, 5041, 5043, 5044, 5046, 5047, 5048, 5056, 5057, 5058, 505

In [18]:
num_improvement = 0
num_worse = 0
average_improvement = 0
average_worse = 0
baseline_scores=[]
adapt_scores=[]

for i, cluster_id in enumerate(range(0,ncluster)):
    target_id_list = cluster_ids[str(cluster_id)]

    Xt = X.iloc[target_id_list]
    yt = df2["gfa"].iloc[target_id_list].values

    Xs = X.drop(target_id_list)
    ys = df2["gfa"].drop(target_id_list).values
    
    src_only = RandomForestClassifier(n_estimators=100, max_depth=10)
    print(f"{i}\t",end="")
    src_only.fit(Xs, ys)

    score = src_only.score(Xt, yt)
    print("src_only score: ", score, end="")
    baseline_scores.append(score)
    
    # Instantiate a KMM model : estimator and target input
    # data Xt are given as parameters with the kernel parameters
    adapt_model = KMM(
        estimator=RandomForestClassifier(n_estimators=200, max_depth=10),
        Xt=Xt,
        kernel="rbf",  # Gaussian kernel
        gamma=1.,     # Bandwidth of the kernel
        verbose=0,
        random_state=0
    )

    # Fit the model.
    adapt_model.fit(Xs, ys)

    # Get the score on target data
    adapt_score = adapt_model.score(Xt, yt)
    print("\tadapt_model score: ", adapt_score)
    adapt_scores.append(adapt_score)
    
    if adapt_score > score:
        num_improvement += 1
        a = (adapt_score/score)-1
        average_improvement += a
    else: 
        num_worse += 1
        w = 1-(adapt_score/score)
        average_worse += w


print(f"average improvement: {(average_improvement/ncluster)*100} %")
print(f"average worse: {(average_worse/ncluster)*100} %")
print(f"baseline avg. accuracy:{sum(baseline_scores)/len(baseline_scores)}")
print(f"adapt avg. accuracy:{sum(adapt_scores)/len(adapt_scores)}")

0	src_only score:  0.9259259259259259	adapt_model score:  0.8888888888888888
1	src_only score:  0.8839285714285714	adapt_model score:  0.8928571428571429
2	src_only score:  0.971830985915493	adapt_model score:  0.9765258215962441
3	src_only score:  0.8857142857142857	adapt_model score:  0.889795918367347
4	src_only score:  0.825503355704698	adapt_model score:  0.8187919463087249
5	src_only score:  0.5537190082644629	adapt_model score:  0.5537190082644629
6	src_only score:  0.6956521739130435	adapt_model score:  0.6902173913043478
7	src_only score:  0.48	adapt_model score:  0.488
8	src_only score:  0.873015873015873	adapt_model score:  0.8809523809523809
9	src_only score:  0.7476635514018691	adapt_model score:  0.7383177570093458
10	src_only score:  0.7391304347826086	adapt_model score:  0.7391304347826086
11	src_only score:  0.6635514018691588	adapt_model score:  0.6542056074766355
12	src_only score:  0.7835051546391752	adapt_model score:  0.7525773195876289
13	src_only score:  0.96039

average improvement: 1.820176693685422 %
average worse: 0.6998283641781646 %
baseline avg. accuracy:0.8077893788356961
adapt avg. accuracy:0.8140024177402045


In [12]:
#!pip3 install adapt
print(f"num_improvement:{num_improvement}")
print(f"num_worse:{num_worse}")


num_improvement:17
num_worse:33


In [13]:
# from sklearn.model_selection import train_test_split


# Xs, Xt, ys, yt = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
'''



# np.random.seed(0)

# # Create source dataset (Xs ~ N(-1, 2))
# # ys = 1 for ys in [-1, 1] else, ys = 0
# Xs = np.random.randn(1000, 1)*2-1
# ys = (Xs[:, 0] > -1.) & (Xs[:, 0] < 1.)

# # Create target dataset (Xt ~ N(1, 2)), yt ~ ys
# Xt = np.random.randn(1000, 1)*2+1
# yt = (Xt[:, 0] > -1.) & (Xt[:, 0] < 1.)

# Instantiate and fit a source only model for comparison
src_only = MLPClassifier(alpha=1, max_iter=1000)
print("TRAIN")
src_only.fit(Xs, ys)

print("src_only score: ", src_only.score(Xt, yt))

# Instantiate a KMM model : estimator and target input
# data Xt are given as parameters with the kernel parameters
adapt_model = KMM(
    estimator=MLPClassifier(alpha=1, max_iter=1000),
    Xt=Xt,
    kernel="rbf",  # Gaussian kernel
    gamma=1.,     # Bandwidth of the kernel
    verbose=0,
    random_state=0
)

# Fit the model.
adapt_model.fit(Xs, ys)

# Get the score on target data
print("adapt_model score: ", adapt_model.score(Xt, yt))

'''

'\n\n\n\n# np.random.seed(0)\n\n# # Create source dataset (Xs ~ N(-1, 2))\n# # ys = 1 for ys in [-1, 1] else, ys = 0\n# Xs = np.random.randn(1000, 1)*2-1\n# ys = (Xs[:, 0] > -1.) & (Xs[:, 0] < 1.)\n\n# # Create target dataset (Xt ~ N(1, 2)), yt ~ ys\n# Xt = np.random.randn(1000, 1)*2+1\n# yt = (Xt[:, 0] > -1.) & (Xt[:, 0] < 1.)\n\n# Instantiate and fit a source only model for comparison\nsrc_only = MLPClassifier(alpha=1, max_iter=1000)\nprint("TRAIN")\nsrc_only.fit(Xs, ys)\n\nprint("src_only score: ", src_only.score(Xt, yt))\n\n# Instantiate a KMM model : estimator and target input\n# data Xt are given as parameters with the kernel parameters\nadapt_model = KMM(\n    estimator=MLPClassifier(alpha=1, max_iter=1000),\n    Xt=Xt,\n    kernel="rbf",  # Gaussian kernel\n    gamma=1.,     # Bandwidth of the kernel\n    verbose=0,\n    random_state=0\n)\n\n# Fit the model.\nadapt_model.fit(Xs, ys)\n\n# Get the score on target data\nprint("adapt_model score: ", adapt_model.score(Xt, yt))\n\n'

In [15]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# weights = adapt_model.predict_weights()
# Xs_weighted = np.random.choice(Xs.ravel(), 1000, p=weights/weights.sum())
# limit_noadapt = src_only.intercept_ / src_only.coef_
# limit_adapt = adapt_model.estimator_.intercept_ / adapt_model.estimator_.coef_
# acc_noadapt = src_only.score(Xt, yt)
# acc_adapt = adapt_model.estimator_.score(Xt, yt)

# k1 = sns.kdeplot(Xs.ravel(), color="blue", bw_method=0.5, label="Source")
# k2 = sns.kdeplot(Xt.ravel(), color="red", bw_method=0.5, label="Target")
# k3 = sns.kdeplot(Xs_weighted, color="blue", ls="--",
#                  bw_method=0.5, label="Reweighted source")

# plt.plot([limit_noadapt[0, 0]]*2, [0, 1], ls=":", c="k")
# plt.plot([limit_adapt[0, 0]]*2, [0, 1], ls="--", c="k")
# plt.fill_between([-1, 1], [0, 0], [1, 1], alpha=0.2, color="k")
# plt.fill_between([-9, 9], [0, 0], [1, 1], alpha=0.05, color="k")
# plt.text(-6, 0.2, "_", fontsize=20)
# plt.text(-0.45, 0.24, "+", fontsize=20)
# plt.text(5.5, 0.2, "_", fontsize=20)
# plt.ylim(0, 0.3); plt.xlim(-9, 9)

# plt.text(-8.5, 0.25,
#          (r"$\cdots$ No adapt : Acc = %.2f"%acc_noadapt + "\n" +
#           r"--- Adapt : Acc = %.2f"%acc_adapt),
#          bbox=dict(boxstyle='round', fc='w', ec="gray"))
# plt.legend(); plt.show();

In [16]:
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline
# from sklearn.datasets import make_moons, make_circles, make_classification
# from sklearn.neural_network import MLPClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.inspection import DecisionBoundaryDisplay

# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1, max_iter=1000),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis(),
# ]